In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Embedding, 
    Conv1D, 
    GlobalMaxPooling1D, 
    Dense, 
    Dropout, 
    SpatialDropout1D, 
    GRU, 
    LSTM, 
    Bidirectional, 
    SimpleRNN, 
    MaxPooling1D, 
    Flatten, 
    BatchNormalization, 
    SeparableConv1D
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Embedding, 
    Conv1D, 
    GlobalMaxPooling1D, 
    Dense, 
    Dropout, 
    SpatialDropout1D, 
    GRU, 
    LSTM, 
    Bidirectional, 
    SimpleRNN, 
    MaxPooling1D, 
    Flatten, 
    BatchNormalization, 
    SeparableConv1D
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Input, Model
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import (
    Embedding, 
    Conv1D, 
    GlobalMaxPooling1D, 
    Dense, 
    Dropout, 
    SpatialDropout1D, 
    GRU, 
    LSTM, 
    Bidirectional, 
    SimpleRNN, 
    MaxPooling1D, 
    Flatten, 
    BatchNormalization, 
    GlobalAveragePooling1D, 
    SeparableConv1D, 
    Attention,
    Concatenate
)
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.regularizers import l2
from keras.optimizers.schedules import ExponentialDecay
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences



In [ ]:
train_data=pd.read_csv("/kaggle/input/bestofbest/best_data.csv")
test_data=pd.read_csv("/kaggle/input/new-dataf/HTS.csv")
del test_data['Unnamed: 0']
del test_data['Index']
# del train_data['Unnamed: 0']


In [ ]:
train_data.dropna(subset=['Event.type'], inplace=True)

# Reset index after dropping rows
train_data.reset_index(drop=True, inplace=True)

In [ ]:
train_data = train_data[train_data['Event.type'] != '-']


In [ ]:

# Assuming train_data and test_data are pandas DataFrame objects

# Combining columns into a new 'combined' column
train_data['combined'] = train_data['Index'] + ' ' + train_data['Lieu'] + ' ' + train_data['Nouvelle.valeur'] + ' ' + train_data['Valeur'] + ' ' + train_data['Ancienne.valeur'] + ' ' + train_data['Date']
# test_data['combined'] = test_data['Index'] + ' ' + test_data['Lieu'] + ' ' + test_data['Nouvelle.valeur'] + ' ' + test_data['Valeur'] + ' ' + test_data['Ancienne.valeur'] + ' ' + test_data['Date']

# Dropping unnecessary columns
train_data = train_data.drop(columns=['Index', 'Lieu', 'Valeur', 'Nouvelle.valeur', 'Ancienne.valeur'])
# test_data = test_data.drop(columns=['Index', 'Lieu', 'Valeur', 'Nouvelle.valeur', 'Ancienne.valeur'])
del train_data['Unnamed: 0']
del train_data['Date']

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, BatchNormalization, Conv1D, MaxPooling1D, SeparableConv1D, Bidirectional, LSTM, GRU, SimpleRNN, Dropout, Dense, GlobalMaxPooling1D, Flatten
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
import pandas as pd
import numpy as np

# Load the data
# df_train = pd.read_csv('/kaggle/input/data-best/NLPT.csv')
# train_data = df_train.drop_duplicates(subset="combined").reset_index(drop=True)
# test_data = pd.read_csv('/kaggle/input/data-best/NLPS.csv')

# Preprocess the data
X = train_data['combined']
y = train_data['Event.type']

# Encode target labels into integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Tokenize and pad sequences
max_words = 100
max_len = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)

X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=max_len)

# Define the CNN model
def create_model():
    model = Sequential(name="Text_Classification_Model")
    model.add(Embedding(input_dim=max_words, output_dim=300, input_length=max_len, name="embedding_layer"))
    model.add(SpatialDropout1D(0.4, name="spatial_dropout"))
    model.add(BatchNormalization(name="batch_norm_"))
    model.add(Conv1D(128, 3, activation='relu', kernel_regularizer=l2(1e-4), padding='same', name="conv1d_1"))
    model.add(BatchNormalization(name="batch_norm_1"))
    model.add(Conv1D(64, 3, activation='relu', kernel_regularizer=l2(1e-4), padding='same', name="conv1d_2"))
    model.add(BatchNormalization(name="batch_norm_2"))
    model.add(Conv1D(64, 3, activation='relu', kernel_regularizer=l2(1e-4), padding='same', name="conv1d_3"))
    model.add(MaxPooling1D(2, name="max_pooling"))
    model.add(SeparableConv1D(64, 3, activation='relu', kernel_regularizer=l2(1e-4), padding='same', name="separable_conv1d"))
    model.add(Bidirectional(LSTM(128, return_sequences=True), name="bidirectional_lstm"))
    model.add(LSTM(128, return_sequences=True, name="cudnn_lstm"))
    model.add(Bidirectional(GRU(128, return_sequences=True), name="bidirectional_gru"))
    model.add(GRU(128, return_sequences=True, name="cudnn_gru"))
    model.add(SimpleRNN(32, return_sequences=True, name="simple_rnn"))
    model.add(Bidirectional(GRU(128, return_sequences=True), name="bidirectnal_gru"))
    model.add(GRU(128, return_sequences=True, name="cnn_gru"))
    model.add(SimpleRNN(32, return_sequences=True, name="smple_rnn"))
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(1e-4), name="dense_1"))
    model.add(BatchNormalization(name="batch_norm_3"))
    model.add(Dropout(0.1, name="dropout"))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(1e-4), name="dense_2"))
    model.add(Dropout(0.2, name="dropout_2"))
    model.add(GlobalMaxPooling1D(name="global_max_pooling"))
    model.add(Flatten(name="flatten"))
    num_classes = 14
    model.add(Dense(num_classes, activation='softmax', name="output_layer"))
    optimizer = RMSprop(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy')
    return model

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Define cross-validation
kfold = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)

# Initialize variables for storing results
cv_scores = []
fold = 1

# Create and compile the model
model = create_model()
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Perform cross-validation
for train_idx, val_idx in kfold.split(X_pad, y_encoded):
    print(f"Fold {fold}")

    # Split data into training and validation sets
    X_train, X_val = X_pad[train_idx], X_pad[val_idx]
    y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]

    # Train the model
    history = model.fit(X_train, y_train, epochs=7, batch_size=256, callbacks=[early_stopping], validation_data=(X_val, y_val))

    # Evaluate the model on validation data
    accuracy = history.history['val_accuracy'][-1]
    cv_scores.append(accuracy)

    fold += 1

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean accuracy:", np.mean(cv_scores))
